In [1]:
import pandas as pd

url = 'https://github.com/mattharrison/datasets/raw/master/data/''vehicles.csv.zip'
df = pd.read_csv(url, dtype_backend='pyarrow', engine='pyarrow')
pd.set_option('display.max_columns', None)

def gt20(val):
    return val > 20

city_mpg = df.city08
highway_mpg = df.highway08
make = df.make

In [2]:
make

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: string[pyarrow]

In [3]:
make.astype(str)

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: object

In [4]:
import pyarrow as pa
string_pa = pd.ArrowDtype(pa.string())
make.astype(str).astype(string_pa)

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: string[pyarrow]

In [5]:
'Ford'.lower()
'Hello'.lower()

'hello'

In [6]:
make.str.lower()

0        alfa romeo
1           ferrari
2             dodge
3             dodge
4            subaru
            ...    
41139        subaru
41140        subaru
41141        subaru
41142        subaru
41143        subaru
Name: make, Length: 41144, dtype: string[pyarrow]

In [7]:
'Alpha Romeo'.find('A')

0

In [8]:
make.str.find('A')

0         0
1        -1
2        -1
3        -1
4        -1
         ..
41139    -1
41140    -1
41141    -1
41142    -1
41143    -1
Name: make, Length: 41144, dtype: int32[pyarrow]

In [9]:
# print(make.str.extract(r'([^a-z A-Z])')) ERROR

In [10]:
print(make.str.extract(r'(?P<non_alpha>[^a-z A-Z])'))

      non_alpha
0          <NA>
1          <NA>
2          <NA>
3          <NA>
4          <NA>
...         ...
41139      <NA>
41140      <NA>
41141      <NA>
41142      <NA>
41143      <NA>

[41144 rows x 1 columns]


In [11]:
(make
 .str.extract(r'(?P<non_alpha>[^a-z A-Z])',
              expand=False)
 .value_counts()
)

non_alpha
-    1727
.      46
,       9
Name: count, dtype: int64[pyarrow]

In [12]:
# If a column in a CSV file contains non-numeric characters, use the following code to find them:

col = make

(col
 .str.extract(r'(?P<non_num>[^0-9.])', expand=False)
 .value_counts()
)

non_num
C    5336
M    4833
F    3686
B    2796
G    2691
D    2679
P    2589
S    2234
T    2159
V    2001
H    1803
A    1610
N    1471
J    1435
L    1241
I     860
K     618
O     462
R     392
E     167
s      38
W      32
Y       8
Q       3
Name: count, dtype: int64[pyarrow]

In [13]:
import pyarrow as pa
string_pa = pd.ArrowDtype(pa.string())
age = pd.Series(['0-10', '11-15', '11-15', '61-65', '46-50'],
                 dtype=string_pa)
age

0     0-10
1    11-15
2    11-15
3    61-65
4    46-50
dtype: string[pyarrow]

In [14]:
age.str.split('-')

0     ['0' '10']
1    ['11' '15']
2    ['11' '15']
3    ['61' '65']
4    ['46' '50']
dtype: list<item: string>[pyarrow]

In [15]:
(age
 .str.split('-', expand=True)
 .iloc[:,0]
 .astype('int8[pyarrow]')
)

0     0
1    11
2    11
3    61
4    46
Name: 0, dtype: int8[pyarrow]

In [16]:
(age
 .str.slice(-2)
 .astype('int8[pyarrow]')
)

0    10
1    15
2    15
3    65
4    50
dtype: int8[pyarrow]

In [17]:
(age
 .str[-2:]
 .astype('int8[pyarrow]')
)

0    10
1    15
2    15
3    65
4    50
dtype: int8[pyarrow]

In [18]:
(age
 .str.split('-', expand=True)
 .astype('int8[pyarrow]')
 .mean(axis='columns')
)

0     5.0
1    13.0
2    13.0
3    63.0
4    48.0
dtype: double[pyarrow]

In [19]:
import random
def between(row):
    return random.randint(*row.values)

In [20]:
(age
 .str.split('-', expand=True)
 .astype(int)
 .apply(between, axis='columns')
)

0     7
1    13
2    11
3    65
4    47
dtype: int64

In [21]:
import numpy as np
print(age
      .str.split('-', expand=True)
      .rename(columns={0:'lower',1:'upper'})
      .astype('int8[pyarrow]')
      .assign(rand=np.random.rand(len(age)),
              age=lambda df_: (df_.lower + (df_.rand *
                                            (df_.upper - df_.lower)))
              .astype('int8[pyarrow]', errors='ignore')
             )
     )


   lower  upper      rand        age
0      0     10  0.990250   9.902499
1     11     15  0.308338   12.23335
2     11     15  0.718090  13.872359
3     61     65  0.014175  61.056701
4     46     50  0.251970  47.007879


In [22]:
import numpy as np # imports numpy
(age # uses the age bins shown here: age = pd.Series(['0-10', '11-15', '11-15', '61-65', '46-50'],
 .str.split('-', expand=True) # splits the bins by '-' into two columns
 .astype(int) # turns the output of the two columns into integers
 .pipe(lambda df_: pd.Series(np.random.randint(df_.iloc[:,0], # not yet sure what pipe does. # takes a random intiger and does something with iloc idk what the comma in iloc[:,0] does
                                               df_.iloc[:,1]), index = df_.index) # idk what [:,1] the comma does here, and does something with an index
      )
)

0     7
1    12
2    14
3    61
4    48
dtype: int64

In [23]:
age_100k = (age # takes age series
            .sample(100_000, replace=True, random_state=42) # creates a randomly sampled figures from age, 100,000 of them
            .reset_index(drop=True) # creates a natural index, as the original index values are kept from the source Series
           )
age_100k

0        61-65
1        46-50
2        11-15
3        46-50
4        46-50
         ...  
99995    11-15
99996     0-10
99997    46-50
99998    61-65
99999     0-10
Length: 100000, dtype: string[pyarrow]

In [24]:
%%timeit
(age_100k
 .str.split('-', expand=True)
 .astype('int8[pyarrow]')
 .pipe(lambda df_: pd.Series(np.random.randint(df_.iloc[:,0], 
                                               df_.iloc[:,1]), index =df_.index)
                            )
      )

11.1 ms ± 192 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%%timeit
(age_100k
 .str.split('-', expand=True)
 .astype('int8[pyarrow]')
 .apply(between, axis='columns')
)

2.13 s ± 28.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%%timeit
(age_100k
 .str.split('-', expand=True)
 .rename(columns={0:'lower', 1:'upper'})
 .astype('int8[pyarrow]')
 .assign(rand=np.random.rand(len(age_100k)),
         age=lambda df_: (df_.lower + (df_.rand * 
                                       (df_.upper - df_.lower)))
         .astype('int8[pyarrow]', errors='ignore')
        )
)

11.6 ms ± 121 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
(age_100k
 .str.split('-', expand=True) # splits into columns
 .rename(columns={0:'lower', 1:'upper'}) # renames the first column as lower, and second as upper
 .astype('int8[pyarrow]') # set type of intiger
 .assign(rand=np.random.rand(len(age_100k)), # creates rand column as decimals for 100 rows contained in age_100k 
         age=lambda df_: (df_.lower + (df_.rand *  ( # the lower age is added to the multiplication of the range differences with the random intiger found in in rand colum
                                       (df_.upper - df_.lower))) # finds the difference between lower and upper
         .astype('int8[pyarrow]', errors='ignore') # converts to int8 using PyArrow and ignores any values that can't be computed
        )
)
)

,lower,upper,rand,age
0,61,65,0.535929,63.143717
1,46,50,0.943963,49.775854
2,11,15,0.952429,14.809717
3,46,50,0.218346,46.873382
4,46,50,0.166949,46.667794
...,...,...,...,...
99995,11,15,0.090197,11.360787
99996,0,10,0.343556,3.435561
99997,46,50,0.183066,46.732263
99998,61,65,0.490661,62.962644


In [71]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [73]:
%%cython
import random
def between_cy(row):
    return random.randint(*row.values)

In [75]:
%%cython
import random
import numpy as np
def between_cy3(x: np.int64, y: np.int64) -> np.int64:
    return random.randint(x,y)

In [79]:
(age # selects age series to run
 .str.split('-', expand=True) # Splits the ages into two collumns
 .astype(int) # converts all to intigers
 .apply(lambda row: between_cy3(row[0], row[1]), axis=1) # apply the between_cy3 function for first and second row, horizontally
)

0     9
1    13
2    14
3    64
4    46
dtype: int64

In [85]:
%%timeit
(age_100k # selects age series to run
 .str.split('-', expand=True) # Splits the ages into two collumns
 .astype(int) # converts all to intigers
 .apply(lambda row: between_cy3(row[0], row[1]), axis=1) # apply the between_cy3 function for first and second row, horizontally
)

399 ms ± 7.72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [97]:
%prun  -l 10 (age_100k.str.split('-', expand=True).astype(int) \
       .apply(lambda row: between_cy3(row[0], row[1]), axis=1))

         4658530 function calls (4658490 primitive calls) in 1.151 seconds

   Ordered by: internal time
   List reduced from 388 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   200000    0.163    0.000    0.575    0.000 series.py:1095(__getitem__)
1100347/1100341    0.101    0.000    0.143    0.000 {built-in method builtins.isinstance}
   100001    0.094    0.000    0.184    0.000 apply.py:1247(series_generator)
   200000    0.091    0.000    0.232    0.000 series.py:1220(_get_value)
   200000    0.072    0.000    0.085    0.000 base.py:3777(get_loc)
   200000    0.064    0.000    0.123    0.000 indexing.py:2765(check_dict_or_set_indexers)
   100000    0.049    0.000    0.778    0.000 <string>:1(<lambda>)
   100000    0.049    0.000    0.129    0.000 random.py:291(randrange)
   100000    0.042    0.000    0.049    0.000 managers.py:2065(set_values)
   100000    0.036    0.000    0.058    0.000 random.py:242(_randbelow_with_getr

In [99]:
%%cython

import numpy as np
import random
def apply_between_cy4(x: np.ndarray[int],
                      y: np.ndarray[int]) -> np.ndarray[int]:
    res: np.ndarray[int] = np.empty(len(x), dtype='int32')
    i : int
    for i in range(len(x)):
        res[i] = random.randint(x[i], y[i])
    return res

Content of stderr:
ld: warning: duplicate -rpath '/opt/anaconda3/lib' ignored

In [101]:
%%timeit
(age_100k
 .str.split('-', expand=True)
 .astype('int8[pyarrow]')
 .pipe(lambda df_: apply_between_cy4(
     df_.iloc[:,0].to_numpy(dtype='int32'),
     df_.iloc[:,1].to_numpy(dtype='int32')))
)

135 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [103]:
%%cython
import numpy as np
import random
def apply_between_cy5(x: np.ndarray[int],
                      y: np.ndarray[int]) -> np.ndarray[long]:
    return np.random.randint(x,y)

Content of stderr:
ld: warning: duplicate -rpath '/opt/anaconda3/lib' ignored

In [105]:
%%timeit
(age_100k
 .str.split('-', expand=True)
 .astype(int)
 .pipe(lambda df_: apply_between_cy5(
     df_.iloc[:,0].to_numpy(dtype='int32'),
     df_.iloc[:,1].to_numpy(dtype='int32')))
 )

23 ms ± 341 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [126]:
# Optimising .apply with Numba

In [128]:
import numba
numba.__version__

'0.59.1'

In [130]:
import numba as nb
import numpy

@nb.jit(nb.int32[:](nb.int32[:], nb.int32[:]))
def between_nb(arr1, arr2):
    return numpy.random.randint(arr1, arr2)

(age_100k
 .str.split('-', expand=True)
 .astype(int)
 .pipe(lambda df_: between_nb(df_.iloc[:,0].to_numpy(dtype='int32'),
                              df_.iloc[:,1].to_numpy(dtype='int32')))
)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in method randint of numpy.random.mtrand.RandomState object at 0x10477a740>) found for signature:
 
 >>> randint(array(int32, 1d, A), array(int32, 1d, A))
 
There are 6 candidate implementations:
[1m      - Of which 2 did not match due to:
      Overload in function 'np_randint_impl_1': File: numba/cpython/randomimpl.py: Line 568.
        With argument(s): '(array(int32, 1d, A), array(int32, 1d, A))':[0m
[1m       Rejected as the implementation raised a specific error:
         TypingError: [1mtoo many positional arguments[0m[0m
  raised from /opt/anaconda3/lib/python3.12/site-packages/numba/core/typing/templates.py:784
[1m      - Of which 2 did not match due to:
      Overload of function 'randint': File: numba/cpython/randomimpl.py: Line 574.
        With argument(s): '(array(int32, 1d, A), array(int32, 1d, A))':[0m
[1m       No match.[0m
[1m   